# GHG Price Sensitivity Analysis

Plot food consumption (kcal/person/day) vs GHG price (USD/tCO2eq) on a logarithmic scale.
Shows how consumption patterns shift as the carbon price increases.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from sensitivity_utils import (
    aggregate_food_groups,
    assign_food_colors,
    extract_consumption_data,
    extract_ghg_data,
    extract_objective_data,
    extract_scenarios_with_param,
    get_log_ticks,
    load_food_to_group,
    plot_objective_sensitivity,
    plot_stacked_sensitivity,
    prepare_objective_data,
)

In [ ]:
# Configuration
CONFIG_NAME = "ghg"
PROJECT_ROOT = Path("..").resolve()
RESULTS_DIR = PROJECT_ROOT / "results" / CONFIG_NAME
PROCESSING_DIR = PROJECT_ROOT / "processing" / CONFIG_NAME

# Load food to group mapping
FOOD_TO_GROUP = load_food_to_group(PROJECT_ROOT)

# Constants
CONSTANT_HEALTH_VALUE_PER_YLL = 10000
CONSTANT_GHG_PRICE = 100
N_WORKERS = 8

In [ ]:
# Extract scenarios from config
scenarios = extract_scenarios_with_param(
    PROJECT_ROOT,
    CONFIG_NAME,
    param_path=["emissions", "ghg_price"],
    scenario_prefix="ghg_",
)

# Filter to only include scenarios with existing network files
scenarios = [(p, s, f) for p, s, f in scenarios if f.exists()]

print(f"Found {len(scenarios)} GHG scenarios:")
for ghg_price, name, _ in scenarios:
    print(f"  {name}: {ghg_price:,.0f} USD/tCO2eq")

# Extract parameter values for tick generation
param_values = [p for p, _, _ in scenarios]

## Food Consumption

In [ ]:
# Extract consumption data
df = extract_consumption_data(
    scenarios,
    FOOD_TO_GROUP,
    RESULTS_DIR / "plots" / "ghg_sensitivity.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

print(f"\nConsumption data shape: {df.shape}")
df

In [ ]:
# Aggregate food groups and prepare for plotting
df_plot = aggregate_food_groups(df)

# Order groups by consumption in the lowest GHG price scenario (descending) for consistent stacking
min_ghg = df_plot.index.min()
group_order = df_plot.loc[min_ghg].sort_values(ascending=False).index.tolist()
df_plot = df_plot[group_order]

# Assign colors
colors = assign_food_colors(df_plot)

print(
    f"Food groups (ordered by ghg_{int(min_ghg)} caloric consumption, bottom to top):"
)
for i, group in enumerate(group_order):
    print(f"  {i}: {group}: {df_plot.loc[min_ghg, group]:.1f} kcal/person/day")

In [ ]:
# X-axis configuration - derive from scenarios
GHG_XTICKS, GHG_XTICKLABELS = get_log_ticks(param_values)
GHG_XLABEL = "GHG price [USD/tCO2eq]"

print(f"X-axis ticks: {GHG_XTICKS}")
print(f"X-axis labels: {GHG_XTICKLABELS}")

# Manual label positions
LABEL_X_POSITIONS = {
    "eggs_poultry": 8,  # Move right to avoid overlap with red_meat
}

In [ ]:
fig, ax = plt.subplots(figsize=(3.54, 2.5))  # 90mm x ~63mm
plot_stacked_sensitivity(
    df_plot,
    colors,
    ax,
    xlabel=GHG_XLABEL,
    ylabel="Food consumption [kcal/person/day]",
    panel_label="a",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    label_x_positions=LABEL_X_POSITIONS,
    y_max=2400,
)
plt.tight_layout()
plt.savefig(RESULTS_DIR / "plots" / "ghg_sensitivity.pdf", dpi=300, bbox_inches="tight")
plt.show()

## Objective Breakdown

Plot objective components (billion USD) vs GHG price on a logarithmic scale.
Health burden is valued at a constant $10,000/YLL for comparability across scenarios.

In [ ]:
# Extract objective data
df_obj = extract_objective_data(
    scenarios,
    RESULTS_DIR / "plots" / "ghg_objective_breakdown.csv",
    param_name="ghg_price",
    constant_health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    constant_ghg_price=CONSTANT_GHG_PRICE,
    n_workers=N_WORKERS,
)

print(f"\nObjective data shape: {df_obj.shape}")
df_obj

In [ ]:
# Prepare objective data
df_obj = prepare_objective_data(df_obj)

print("Objective categories (ordered):")
for cat in df_obj.columns:
    sign = "+" if df_obj[cat].mean() > 0 else "-"
    print(f"  {sign} {cat}: {df_obj[cat].mean():.1f} billion USD (mean)")

In [ ]:
fig, ax = plt.subplots(figsize=(3.54, 2.5))  # 90mm x ~63mm
plot_objective_sensitivity(
    df_obj,
    ax,
    xlabel=GHG_XLABEL,
    panel_label="c",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    ghg_price=CONSTANT_GHG_PRICE,
    highlight_cat="Health burden",
)
plt.tight_layout()
plt.savefig(
    RESULTS_DIR / "plots" / "ghg_objective_breakdown.pdf", dpi=300, bbox_inches="tight"
)
plt.show()

## GHG Emissions by Food Group

Plot GHG emissions (GtCO2eq) by food group vs GHG price on a logarithmic scale.
Uses flow-based attribution to trace emissions from consumption back through trade and processing to production.

In [ ]:
# Extract GHG data
df_ghg = extract_ghg_data(
    scenarios,
    FOOD_TO_GROUP,
    RESULTS_DIR / "plots" / "ghg_ghg_by_food_group.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

print(f"\nGHG data shape: {df_ghg.shape}")
min_ghg_price = df_ghg.index.min()
print(
    f"Total GHG at ghg_{int(min_ghg_price)}: {df_ghg.loc[min_ghg_price].sum():.2f} GtCO2eq"
)
df_ghg

In [ ]:
# Aggregate and use same order as panel a
df_ghg_plot = aggregate_food_groups(df_ghg)
available_groups = [g for g in group_order if g in df_ghg_plot.columns]
df_ghg_plot = df_ghg_plot[available_groups]

print("Food groups (same order as panel a, based on consumption):")
min_ghg_price = df_ghg_plot.index.min()
for group in available_groups:
    print(f"  {group}: {df_ghg_plot.loc[min_ghg_price, group]:.3f} GtCO2eq")

In [ ]:
# Labels to skip in the GHG emissions plot
GHG_LABEL_SKIP = {"legumes", "fruits_vegetables"}

In [ ]:
fig, ax = plt.subplots(figsize=(3.54, 2.5))  # 90mm x ~63mm
plot_stacked_sensitivity(
    df_ghg_plot,
    colors,
    ax,
    xlabel=GHG_XLABEL,
    ylabel="GHG emissions [GtCO2eq]",
    panel_label="b",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    label_skip=GHG_LABEL_SKIP,
    min_height_for_label=0.08,
)
plt.tight_layout()
plt.savefig(
    RESULTS_DIR / "plots" / "ghg_ghg_by_food_group.pdf", dpi=300, bbox_inches="tight"
)
plt.show()

## Combined Multipanel Figure

Create a 2x2 figure combining all three plots with panel d reserved for future use.

In [ ]:
# Create 2x2 multipanel figure
fig, axes = plt.subplots(
    2, 2, figsize=(7.08, 5.0)
)  # ~180mm x ~127mm (two-column width)

# Plot a: Food consumption (top-left)
plot_stacked_sensitivity(
    df_plot,
    colors,
    axes[0, 0],
    xlabel=GHG_XLABEL,
    ylabel="Food consumption [kcal/person/day]",
    panel_label="a",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    label_x_positions=LABEL_X_POSITIONS,
    y_max=2400,
)

# Plot b: GHG emissions (top-right)
plot_stacked_sensitivity(
    df_ghg_plot,
    colors,
    axes[0, 1],
    xlabel=GHG_XLABEL,
    ylabel="GHG emissions [GtCO2eq]",
    panel_label="b",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    label_skip=GHG_LABEL_SKIP,
    min_height_for_label=0.08,
)

# Plot c: Objective breakdown (bottom-left)
plot_objective_sensitivity(
    df_obj,
    axes[1, 0],
    xlabel=GHG_XLABEL,
    panel_label="c",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    ghg_price=CONSTANT_GHG_PRICE,
    highlight_cat="Health burden",
)

# Panel d: Empty for now (bottom-right)
axes[1, 1].axis("off")
axes[1, 1].text(
    0.5,
    0.5,
    "d",
    transform=axes[1, 1].transAxes,
    fontsize=9,
    fontweight="bold",
    va="center",
    ha="center",
    alpha=0.3,
)

plt.tight_layout()
plt.savefig(
    RESULTS_DIR / "plots" / "ghg_sensitivity_combined.pdf", dpi=300, bbox_inches="tight"
)
plt.show()